# Recomendador simple
Recomendar aquello que consideremos "lo mejor". 

#### Principales caracteristicas
- Buenos resultados para lo que suele interesar a la mayoría
- Se suele usar como *baseline* para comparar con otros modelos
- No personaliza, no se adapta a los usuarios con intereses especificos, que es un de los principales objetivos de los sistemas de recomendación

#### Como lo hacemos
1. Definir una métrica para definir el interes
2. Calcular la métrica para todos los elemento
3. Ordenar por la métrica y seleccionar los mejores


# Antes de comenzar... veamos el dataset

Contenidos audiovisuales disponibles en la plataforma de Movistar+ 

In [1]:
import pandas as pd

dsMP = pd.read_csv('./dataset/Contenidos_movistar.csv', sep='|')
dsMP.head(5)

,_id,nombre_ws,idFuente,nombreFuente,numeroValoraciones,valoracion,valoracionMaxima
0,a3770283-44c9-4573-913a-ef360574ffaf,El padrino. Parte II,1,movistarplus,324,4.4,5
1,800f26aa-4a6f-4bb6-937d-0ebf968eb461,El caballero oscuro,1,movistarplus,1962,4.6,5
2,b98d82f7-1263-4fb5-9f38-dac2605362db,Forrest Gump,1,movistarplus,131,3.5,5
3,a5363720-ab0d-41b4-8337-4b9ff4fc0be4,El señor de los anillos: La comunidad del anillo,1,movistarplus,531,4.4,5
4,4b4302a7-27b1-4077-9c72-cdfa6af28398,Cadena perpetua,1,movistarplus,57,3.2,5


# Creación de la métrica

**OPCIÓN 1** Usar el campo valoración de los usarios. Pero que ocurre en este caso:

> Película X: valoración 4.9, numeroValoracion=3


> Película Y: valoración 4.5, numeroValoracion = 800

Sería mejor la película X que la Y, aun cuando esta la han valorado pocos usuarios

In [2]:
dsMP.sort_values('valoracion', ascending=False).head(20)

,_id,nombre_ws,idFuente,nombreFuente,numeroValoraciones,valoracion,valoracionMaxima
16202,648b5a3d-55a2-4e25-a238-8dd531f340f4,Transforma digital,1,movistarplus,1,5.0,5
24947,a0dbd3e1-070c-4aba-992a-f60ca121e7e8,ICarly,1,movistarplus,1,5.0,5
7672,fb487d30-124b-42ec-9f49-95391fb92b77,El chino,1,movistarplus,1,5.0,5
18217,7745fd5c-47e6-4df9-9b4d-15b407e987d9,MTV Movie and TV Awards 2018,1,movistarplus,2,5.0,5
11173,0f67bfc0-05bd-4cdb-8668-243c44368ea2,Rally Dakar: Resumen,1,movistarplus,1,5.0,5
19320,07b4f7a4-e811-49d2-b21c-6c25a88e23e4,Camping 2,1,movistarplus,1,5.0,5
7541,9b49eed9-1f72-40e6-9534-b9c2ab226911,Unidas por una traición,1,movistarplus,3,5.0,5
16925,cae9bb45-22ce-459a-b6f3-dccfcc4ded0a,La noche de Cadena 100,1,movistarplus,1,5.0,5
11178,1a0111f0-5224-425c-ba37-7fb6d00bfd11,"Luge. Copa del mundo: Konigsee, segunda manga ...",1,movistarplus,1,5.0,5
26877,3898fbfb-8eab-4091-9f35-3ad6546f0e70,Full Frontal with Samantha Bee,1,movistarplus,1,5.0,5


**OPCIÓN 2** Usar una valoración ponderada

<img src="./imagenes/score.png">

donde,
- *n* es el número de valoraciones
- *n_min* es el mínimo de votos requeridos
- *V* es la valoración del recurso
- *M* es la valoración media de todos los recursos

Un posible valor para n_min puede ser el percentil 90, por lo que no se tendrian en cuanta aquellas que han tenido un 90% de votos que el resto

In [3]:
#Cálculo de lo valores
n_min =dsMP['numeroValoraciones'].quantile(0.9)
M=dsMP['valoracion'].mean()
print ('n_min = ' + str(n_min))
print ('M = ' + str(M))
#Se usa copy para evitar referencia y que sea una nueva variable
dsMPFiltrado= dsMP.copy().loc[dsMP['numeroValoraciones']>n_min]
print ('Tamaño dataset original ' + str(dsMP.shape))
print ('Tamaño dataset filtrado ' + str(dsMPFiltrado.shape))


n_min = 191.0
M = 3.2095094983674883
Tamaño dataset original (26952, 7)
Tamaño dataset filtrado (2678, 7)


In [4]:
def calcularScore(n, n_min, V, M):
    return (n/(n+n_min) * V) + (n_min/(n+n_min)*M)

dsMPFiltrado['score']=dsMPFiltrado.apply(lambda X : calcularScore(X['numeroValoraciones'], n_min, X['valoracion'], M), axis=1)
dsMPFiltrado =dsMPFiltrado.sort_values('score', ascending=False)
dsMPFiltrado.head(20)

,_id,nombre_ws,idFuente,nombreFuente,numeroValoraciones,valoracion,valoracionMaxima,score
24108,f8b0c5d2-de41-4c37-954b-1bef1df3eca6,La Resistencia,1,movistarplus,2455,4.7,5,4.592410
89,3fd34760-855d-44d9-9aa2-9e8f17d84510,Zootrópolis,1,movistarplus,4227,4.6,5,4.539886
1,800f26aa-4a6f-4bb6-937d-0ebf968eb461,El caballero oscuro,1,movistarplus,1962,4.6,5,4.476645
10057,878502a5-ba62-482f-87a1-48b3132ef730,Breaking Bad,1,movistarplus,6159,4.5,5,4.461184
10178,0a9a0ab6-11e4-46e7-ab89-6f3e22a2d087,Peaky Blinders,1,movistarplus,1312,4.6,5,4.423298
4359,abe16956-4bf1-4bd5-bb8f-697272a6e414,Coco,1,movistarplus,1080,4.6,5,4.391044
14245,30c8b91f-8480-4eec-bfa6-f10f3fee36c5,Riverdale,1,movistarplus,2041,4.5,5,4.389568
6763,6d60247e-160c-4cf3-a28f-54439f74d596,Figuras ocultas,1,movistarplus,1025,4.6,5,4.381592
12,a391ff73-8530-4c7b-aff4-99bfce7795bb,La guerra de las galaxias. Episodio V: El impe...,1,movistarplus,1598,4.5,5,4.362223
10901,9197429c-f767-4de0-8ed5-aa6cf263c2df,Shameless,1,movistarplus,5669,4.4,5,4.361197
